In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import xml.etree.ElementTree as ET

In [2]:
t = pd.read_json("./dataSite.json")

In [3]:
t

,name,GoodsCode,PlaceCode,link
0,"Final (11.19)Nov 19, 2023",23010160,23000829,https://www.globalinterpark.com/product/23010160
1,"Semi Finals (11.12)Nov 12, 2023",23009896,23000883,https://www.globalinterpark.com/product/23009896
2,"Semi Finals (11.11)Nov 11, 2023",23009895,23000883,https://www.globalinterpark.com/product/23009895
3,"Swiss Stage (10.29)Oct 29, 2023",23009642,23000822,https://www.globalinterpark.com/product/23009642
4,"Swiss Stage (10.22)Oct 22, 2023",23009338,23000822,https://www.globalinterpark.com/product/23009338
5,"Swiss Stage (10.20)Oct 20, 2023",23009336,23000822,https://www.globalinterpark.com/product/23009336
6,"Swiss Stage (10.19)Oct 19, 2023",23009335,23000822,https://www.globalinterpark.com/product/23009335
7,"Swiss Stage (10.27)Oct 27, 2023",23009640,23000822,https://www.globalinterpark.com/product/23009640
8,"Swiss Stage (10.21)Oct 21, 2023",23009337,23000822,https://www.globalinterpark.com/product/23009337
9,"Swiss Stage (10.28)Oct 28, 2023",23009641,23000822,https://www.globalinterpark.com/product/23009641


In [4]:
def xml_to_dataframe(xml_text):
    try:
        root = ET.fromstring(xml_text)
        
        data = []
        for element in root:
            row = {}
            for child in element:
                row[child.tag] = child.text
            data.append(row)
        
        df = pd.DataFrame(data)
        return df
    
    except Exception as e:
        print(f"Erreur lors de la conversion XML en DataFrame : {str(e)}")
        return None

def formatName(name):
    name = name.split(" - ")[-1].split(" ~ ")[0]
    name = name.replace("스위스 스테이지", "Swiss Stage")
    name = name.replace("플레이-인 스테이지", "Play-in")
    return name

def getPlaceCode(GoodsCode):
    res = get(f"https://ticket.globalinterpark.com/Global/Play/CBT/GoodsInfo_CBT.asp?GoodsCode={GoodsCode}&lang=en&gatetype=CBT&device=pc&memNo=")

    for line in res.text.split("\n"):
        if line.__contains__('SetData("PlaceCode"'):
            return line.split('"')[-2]

def getSiteInfos():
    url = "https://www.globalinterpark.com/search-list?q=2023+League+of+Legends+World+Championship+&lang=ko"
    response = get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    dataSite = {"name":[], "GoodsCode":[], "PlaceCode":[], "link":[]}

    events = soup.findAll("li")

    for e in events:
        if e.text.startswith("2023 리그 오브 레전드 월드 챔피언십"):
            dataSite["name"].append(formatName(e.text))
            dataSite["GoodsCode"].append(e.a['href'].split("/")[-1])
            dataSite["PlaceCode"].append(getPlaceCode(dataSite["GoodsCode"][-1]))
            dataSite["link"].append(f"https://www.globalinterpark.com{e.a['href']}")

    dataSite = pd.DataFrame.from_dict(dataSite)
    dataSite.to_json("dataSite.json")

def getPlaceInfos(GoodsCode, PlaceCode):
    res = get(f"https://ticket.globalinterpark.com/Global/Play/Goods/GoodsInfoXml.asp?Flag=RemainSeat&GoodsCode={GoodsCode}&PlaceCode={PlaceCode}&PlaySeq=001&LanguageType=G2001")
    return xml_to_dataframe(res.text)

def getTicketInfos():
    try:
       dataSite = pd.read_json("dataSite.json")
    except:
        getSiteInfos()
        dataSite = pd.read_json("dataSite.json")

    dataSite["SeatInfos"] = dataSite.apply(lambda x: getPlaceInfos(x["GoodsCode"], x["PlaceCode"]), axis=1)
    dataSite["isFull"] = dataSite.apply(lambda x: x["SeatInfos"]["RemainCnt"].astype(int).sum() == 0, axis=1)

    return dataSite

def getAvailableSeats(df):
    seatInfos = df["SeatInfos"]

    infos = seatInfos[seatInfos["RemainCnt"].astype(int) >= 2][["SeatGradeName", "RemainCnt", "SalesPrice"]]
    
    if len(infos) > 0:
        return df["name"], infos, df["link"]
    else:
        return None

In [5]:
dataSite = getTicketInfos()

In [7]:
filterData = dataSite[(dataSite["name"].str.contains("11.3")) & (~dataSite["isFull"])]

In [8]:
for row in filterData.iterrows():
        info = getAvailableSeats(row[1])
        print(info)

('Quarter Finals (11.3)Nov 03, 2023',   SeatGradeName RemainCnt SalesPrice
1         TIER2         2     125000
2         TIER3         5     110000
3         TIER4         4      95000
5         TIER6         9      65000
6         TIER7         7      50000, 'https://www.globalinterpark.com/product/23009892')


In [28]:
info[1][ (info[1]["SeatGradeName"].isin([f"TIER{t}" for t in range(3, 6)])) & (info[1]["RemainCnt"].astype(int) > 1)]

,SeatGradeName,RemainCnt,SalesPrice
2,TIER3,5,110000
3,TIER4,4,95000


In [21]:
info[1]["RemainCnt"].astype(int) > 2

1    False
2     True
3     True
5     True
6     True
Name: RemainCnt, dtype: bool

In [27]:
info[2]

'https://www.globalinterpark.com/product/23009892'